In [145]:
import pandas as pd
import numpy as np

In [190]:
gu_change=pd.read_csv('data/15.상권분석서비스(자치구별 상권변화지표).csv', encoding='utf-8')
df0 = pd.read_csv('data/00.상권분석서비스(상권-영역).csv', encoding='utf-8')

In [24]:
gu_change.columns

Index(['기준_년_분기_코드', '시군구_코드', '시군구_코드_명', '상권_변화_지표', '상권_변화_지표_명',
       '운영_영업_개월_평균', '폐업_영업_개월_평균', '서울_운영_영업_개월_평균', '서울_폐업_영업_개월_평균'],
      dtype='object')

In [191]:
df0.columns

Index(['상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '엑스좌표_값', '와이좌표_값',
       '시군구_코드', '행정동_코드'],
      dtype='object')

In [219]:
gu_name=gu_change[['시군구_코드','시군구_코드_명']].drop_duplicates()
gu_name_en=gu_name.rename(columns={'시군구_코드':'guCode','시군구_코드_명':'guName'})

In [220]:
gu_name_en.to_csv('gu_name.csv', index=False)

In [207]:
gu_t1=gu_change[['시군구_코드','운영_영업_개월_평균', '폐업_영업_개월_평균','상권_변화_지표_명']]
gu_t1=gu_t1.replace('상권확장',1).replace('다이나믹' ,4).replace('정체',2).replace('상권축소',3)

In [123]:
gu_t1.columns

Index(['시군구_코드', '운영_영업_개월_평균', '폐업_영업_개월_평균', '서울_운영_영업_개월_평균',
       '서울_폐업_영업_개월_평균', '상권_변화_지표_명'],
      dtype='object')

In [124]:
gu_t1.rename(columns={'시군구_코드':'guCode', '운영_영업_개월_평균':'openMonthMean', '폐업_영업_개월_평균':'closeMonthMean', '서울_운영_영업_개월_평균':'seoulOpenMonthMean',
       '서울_폐업_영업_개월_평균':'seoulCloseMonthMean', '상권_변화_지표_명':'guChangPoint'})

,guCode,openMonthMean,closeMonthMean,seoulOpenMonthMean,seoulCloseMonthMean,guChangPoint
0,11740,106,50,110,54,4
1,11710,108,54,110,54,1
2,11680,102,50,110,54,4
3,11650,118,54,110,54,2
4,11620,108,52,110,54,4
...,...,...,...,...,...,...
633,11215,89,44,91,44,4
634,11200,93,46,91,44,2
635,11170,101,47,91,44,2
636,11140,114,49,91,44,2


In [138]:
# gu_t1.info()

In [154]:
gu_t1['운영_영업_개월_평균'].min() , gu_t1['운영_영업_개월_평균'].max()
gu_t1['폐업_영업_개월_평균'].min() , gu_t1['폐업_영업_개월_평균'].max()
# gu_t1['서울_운영_영업_개월_평균'].unique()
# gu_t1['운영_영업_개월_평균'].unique()
# gu_t1['운영_영업_개월_평균'].unique()

(41, 63)

In [118]:
# gu_t1

In [125]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [126]:
X=gu_t1.iloc[:,:-1]
y=gu_t1.iloc[:,-1:]

In [127]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
#                                                     test_size=0.3, 
                                                    stratify=y,
                                                    random_state=42)

In [128]:
# DecisionTreeClassifier 객체 생성
dt_clf = DecisionTreeClassifier(random_state=42)

# 학습수행
dt_clf.fit(X_train, y_train)

dt_clf.score(X_train, y_train), dt_clf.score(X_test, y_test)

(0.99581589958159, 0.96875)

In [129]:
# 학습이 완료된 DecisionTreeClassifier 객체에서 테스트 데이터 세트로 예측 수행
y_pred = dt_clf.predict(X_test)

In [130]:
from sklearn.metrics import accuracy_score
print('예측정확도 : {0: .4f}'.format(accuracy_score(y_test, y_pred)))

예측정확도 :  0.9688


In [131]:
from sklearn.model_selection import cross_val_score

cross_val_score(dt_clf, X_test, y_test, cv=10).mean()

0.9125

In [132]:
###############################

In [133]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 학습 진행
forest = RandomForestClassifier(n_estimators=100, max_depth=10)
forest.fit(X_train, y_train)

# 예측
y_pred = forest.predict(X_test)
print(y_pred)
print(list(y_test))

#Random forest 정확도 측정
print(forest.score(X_train, y_train),forest.score(X_test, y_test))


# 정확도 확인
# accuracy_score(실제 레이블 세트, 예측 레이블 세트) : 정확도를 측정하기 위한 사이킷런의 함수
print('정확도 :', accuracy_score(y_test, y_pred))

from sklearn.model_selection import cross_val_score
cross_val_score(forest, X_test, y_test, cv=10).mean()

C:\ProgramData\Miniconda3\envs\hoseo\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


[4 4 3 3 4 4 4 4 4 4 4 4 4 4 4 2 2 4 1 4 4 4 2 2 4 2 4 3 2 2 4 4 4 4 1 2 1
 4 1 4 1 4 4 4 4 2 3 2 4 4 2 4 4 2 2 4 3 2 2 2 4 4 4 2 2 4 4 3 4 2 4 4 4 2
 4 2 4 4 2 2 3 4 3 4 2 4 4 2 2 2 2 4 4 1 1 4 4 4 2 4 2 4 2 4 2 4 4 2 3 4 4
 2 4 2 4 3 4 2 4 2 2 4 4 2 4 4 1 4 4 2 2 2 2 4 2 4 4 4 3 2 4 4 3 4 2 4 4 4
 4 4 4 2 4 2 2 4 3 4 4 1]
['상권_변화_지표_명']
0.99581589958159 0.975
정확도 : 0.975


C:\ProgramData\Miniconda3\envs\hoseo\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Miniconda3\envs\hoseo\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Miniconda3\envs\hoseo\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Miniconda3\envs\hoseo\lib\site-packages\sklearn\model_selection\_validation.py:

0.9125

In [134]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=4,max_depth=10)
gbc.fit(X_train, y_train)

print(gbc.score(X_train, y_train), gbc.score(X_test, y_test))

from sklearn.model_selection import cross_val_score
cross_val_score(gbc, X_test, y_test, cv=10).mean()

C:\ProgramData\Miniconda3\envs\hoseo\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Miniconda3\envs\hoseo\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Miniconda3\envs\hoseo\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Miniconda3\envs\hoseo\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samp

0.99581589958159 0.93125


C:\ProgramData\Miniconda3\envs\hoseo\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Miniconda3\envs\hoseo\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Miniconda3\envs\hoseo\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Miniconda3\envs\hoseo\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samp

0.85

In [136]:
import joblib
joblib.dump(forest, 'sg-model.pkl')

['sg-model.pkl']

In [176]:
df0=pd.read_csv('data/00.상권분석서비스(상권-영역).csv', encoding='utf-8')
df0.columns
# df0

Index(['상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '엑스좌표_값', '와이좌표_값',
       '시군구_코드', '행정동_코드'],
      dtype='object')

In [192]:
df18=pd.read_csv('data/18.상권분석서비스(상권-점포).csv', encoding='utf-8')
# df18

In [178]:
df18=df18.merge(df0, on='상권_코드').drop_duplicates()

In [179]:
df18.columns
df18_t=df18[['기준_년_분기_코드', '상권_코드', '서비스_업종_코드','점포_수','시군구_코드']]

In [180]:
df18_t2=df18_t.groupby(['기준_년_분기_코드','시군구_코드']).sum().reset_index()

In [122]:
# df18_t2

In [195]:
import pandas as pd
import numpy as np

In [196]:
df21 = pd.read_csv('data/21.상권분석서비스(상권-추정유동인구).csv' , encoding='utf-8')
df21 = df21.drop(df21.filter(regex='연령대|시간대'),axis=1)
# df21

In [197]:
target3=['월요','화요','수요','목요','금요']
target4=['토요','일요']
def add_week2(df,target3,target4):
    df['주중_유동인구_수']=df.filter(regex='|'.join(target3)).sum(axis=1)
    df['주말_유동인구_수']=df.filter(regex='|'.join(target4)).sum(axis=1)
    df_a=df.drop(df.filter(regex='|'.join(target3+target4)),axis=1)
    return df_a

In [198]:
df21.columns

Index(['기준 년코드', '기준_분기_코드', ' 상권_구분_코드', ' 상권_구분_코드_명', '상권_코드', '상권_코드_명',
       '총_유동인구_수', '남성_유동인구_수', '여성_유동인구_수', '월요일_유동인구_수', '화요일_유동인구_수',
       '수요일_유동인구_수', '목요일_유동인구_수', '금요일_유동인구_수', '토요일_유동인구_수', '일요일_유동인구_수'],
      dtype='object')

In [199]:
df21=add_week2(df21,target3,target4)
# df21['기준_년_분기_코드'] = 
df21['기준_년_분기_코드']=df21['기준 년코드'].astype('str')+'_'+df21['기준_분기_코드'].astype('str')
df21.columns

Index(['기준 년코드', '기준_분기_코드', ' 상권_구분_코드', ' 상권_구분_코드_명', '상권_코드', '상권_코드_명',
       '총_유동인구_수', '남성_유동인구_수', '여성_유동인구_수', '주중_유동인구_수', '주말_유동인구_수',
       '기준_년_분기_코드'],
      dtype='object')

In [200]:
df21_1=df21[[ '기준_년_분기_코드','상권_코드','총_유동인구_수', '남성_유동인구_수', '여성_유동인구_수', '주중_유동인구_수', '주말_유동인구_수']]

In [203]:
df21_1=df21_1.merge(df0 , on='상권_코드').drop_duplicates()
df21_1.columns

Index(['기준_년_분기_코드', '상권_코드', '총_유동인구_수', '남성_유동인구_수', '여성_유동인구_수',
       '주중_유동인구_수', '주말_유동인구_수', '상권_구분_코드', '상권_구분_코드_명', '상권_코드_명', '엑스좌표_값',
       '와이좌표_값', '시군구_코드', '행정동_코드'],
      dtype='object')

In [206]:
flow_pop=df21_1[['기준_년_분기_코드', '시군구_코드','총_유동인구_수', '남성_유동인구_수', '여성_유동인구_수','주중_유동인구_수', '주말_유동인구_수']]

In [210]:
flow_pop=flow_pop.merge(gu_change, on=['기준_년_분기_코드', '시군구_코드']).drop_duplicates()

In [216]:
flow_pop.columns
gu_pop=flow_pop[['기준_년_분기_코드', '시군구_코드', '총_유동인구_수', '남성_유동인구_수', '여성_유동인구_수','주중_유동인구_수', 
                  '주말_유동인구_수', '시군구_코드_명','운영_영업_개월_평균', '폐업_영업_개월_평균', '서울_운영_영업_개월_평균', '서울_폐업_영업_개월_평균',
                 '상권_변화_지표_명']]

In [217]:
gu_pop.to_csv('flow_pop_ch.csv',index=False )

In [221]:
gu_pop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33340 entries, 0 to 33361
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   기준_년_분기_코드      33340 non-null  object
 1   시군구_코드          33340 non-null  int64 
 2   총_유동인구_수        33340 non-null  int64 
 3   남성_유동인구_수       33340 non-null  int64 
 4   여성_유동인구_수       33340 non-null  int64 
 5   주중_유동인구_수       33340 non-null  int64 
 6   주말_유동인구_수       33340 non-null  int64 
 7   시군구_코드_명        33340 non-null  object
 8   운영_영업_개월_평균     33340 non-null  int64 
 9   폐업_영업_개월_평균     33340 non-null  int64 
 10  서울_운영_영업_개월_평균  33340 non-null  int64 
 11  서울_폐업_영업_개월_평균  33340 non-null  int64 
 12  상권_변화_지표_명      33340 non-null  object
dtypes: int64(10), object(3)
memory usage: 3.6+ MB
